In [3]:
#The purpose of this notebook is to compute the basin stability of a network governed by the coupled oscillator model
#Import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.integrate
import scipy.stats
from scipy.linalg import circulant
from scipy.optimize import curve_fit
import time
import networkx as nx



In [246]:
########################
### DEFINE FUNCTIONS ###
########################

#Generate all phase and natural frequency initial conditions we want to test 
def create_ensembles(T, N, θ_s, num_arrays):

    T_values = np.random.uniform(0,np.pi,T)
    phase_values = []
    omega_values = []
    ensembles=[]
    seen = set() # Set to track unique pairs (for ensuring uniqueness)

    # Create phase ensembles
    for i in range(N):
        phase_list = []
        for entry in T_values:
            phase = [θ_s] * N  # All values set to synchronous state
            phase[i] = entry  # Set one specific position to entry value
            phase_values.append(phase)

    #Create omega ensembles 
    while len(omega_values) < num_arrays:
        arrays = np.ones(N)
        num_elements_to_flip = N // 2  # Number of elements to flip to -1
        indices_to_flip = np.random.choice(np.arange(N), size=num_elements_to_flip, replace=False)  # Randomly select indices to flip
        arrays[indices_to_flip] = -1  # Flip the selected indices

        # Convert the array to a tuple to ensure it's hashable
        arrays_tuple = tuple(arrays)

    # Check if the generated array for omega is unique
        if arrays_tuple not in seen:
            seen.add(arrays_tuple)  # Add to seen set
            omega_values.append(arrays)  # Add the unique array to omega_values
   
    for phase in phase_values:
        for omega in omega_values:
            ensembles.append([phase, omega])  # Create a unique key to represent the combination

                
    return ensembles



#Our model
def kuramoto_function(phase,t,K,N,omega):
    phase_matr = np.tile(phase,(N,1))
    temp = np.multiply(K,np.sin(phase_matr-phase_matr.T))
    incr = omega + np.sum(temp,axis=1)
    return (incr)


#Simulating our model
def simulate_kuramoto(kuramoto, initial_θ, initial_ω, t):
    sol =scipy.integrate.odeint(kuramoto,y0=initial_θ,t=t,args=(K_ij,N,initial_ω),hmax=0.1,full_output=0,rtol=1e-4,atol=1e-4)
    last_value= sol[-1]
    ode_sol = kuramoto_function(sol[-1],0,K_ij,N,initial_ω)
    return sol, np.round(last_value, 6), np.round(ode_sol, 6)

#Computing the basin stability for each node
def basin_stability(arrays, T, num_arrays):
    sim_size_per_node = T * num_arrays

    def all_elements_equal(arr):     # Function to check if all elements in an array are equal
        return np.all(arr == arr[0])  # Check if all elements are the same as the first one

    # Split the list of arrays into chunks of specified size
    chunks = [arrays[i:i + sim_size_per_node] for i in range(0, len(arrays), sim_size_per_node)]

    # Count the number of arrays with all elements equal in each chunk
    U = [sum(all_elements_equal(arr) for arr in chunk) for chunk in chunks]
    basin_stability = [x /sim_size_per_node  for x in U]


    return basin_stability

    


In [247]:

##########################
#### MODEL PARAMETERS #### 
##########################

T= 2 # number of test ensembles
N= 3 #number of nodes


#Time iterations for ODE solver
t_max = 10000
t_min = 0
t = np.arange(t_min,t_max,1)

#Create Network
G=nx.complete_graph(N)
K_ij=nx.to_numpy_array(G)
#nx.draw(G)

In [251]:

##########################
##### RUN SIMULATION #####
##########################


simulation_results = []
ensembles= create_ensembles(T, N, 1, 3)


#Run simulation
for phase, omega in ensembles:
    # Simulate the Kuramoto model
    result = simulate_kuramoto(
        kuramoto_function,
        initial_θ=phase,
        initial_ω=omega,
        t=t
    )
    
    # Store results
    simulation_results.append(result)

frequency = [ODE_sol[2] for ODE_sol in simulation_results]

In [252]:
node_bs= basin_stability(frequency, T, 3)